In [39]:
import urllib
from sqlalchemy import create_engine

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost\\SQLEXPRESS;"       # Change this if needed
    "DATABASE=master;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
    "Encrypt=no;"           
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


In [41]:
from sqlalchemy import text

try:
    with engine.connect() as conn:
        print("✅ Connected to SQL Server successfully!")

        result = conn.execute(text("SELECT GETDATE() AS CurrentTime;"))

        for row in result:
            print("🕒 Server Time:", row.CurrentTime)
except Exception as e:
    print("❌ Connection failed.")
    print("Error:", e)


✅ Connected to SQL Server successfully!
🕒 Server Time: 2025-07-30 03:00:24.633000


In [43]:
import pandas as pd

# Load data from the ExamScores table
with engine.connect() as conn:
    df = pd.read_sql("SELECT * FROM ExamScores", conn)

print(df.head())

   StudentID  Subject  CA_Score  Mock_Score  Final_Score  External_Score  \
0        101    Maths        65          68           70              72   
1        102  English        55          60           58              59   
2        103  Biology        70          75           72              74   
3        104    Maths        50          52           54              53   
4        105  English        60          62           64              65   

   Attendance  StudyHours  Malpractice  
0        90.0        12.0          0.0  
1        85.0        10.5          0.0  
2        95.0        14.0          0.0  
3        80.0         8.0          0.0  
4        92.0        13.5          0.0  


In [45]:
import numpy as np

df['Malpractice_Label'] = np.where(
    (df['External_Score'] > df[['CA_Score', 'Mock_Score', 'Final_Score']].mean(axis=1) + 25),
    1,
    0
)


In [47]:
features = ['CA_Score', 'Mock_Score', 'Final_Score', 'External_Score']
X = df[features]
y = df['Malpractice_Label']


In [49]:
df.to_sql('ExamScoresLabeled', engine, if_exists='replace', index=False)


20

In [51]:

print(df['Malpractice_Label'].isnull().sum())


0


In [53]:
print(df['Malpractice_Label'].value_counts())


Malpractice_Label
0    10
1    10
Name: count, dtype: int64


In [55]:
X = df[['CA_Score', 'Mock_Score', 'Final_Score', 'External_Score']]
y = df['Malpractice_Label']

# continue with train_test_split and model


In [57]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features and target
features = ['CA_Score', 'Mock_Score', 'Final_Score', 'External_Score']
X = df[features]
y = df['Malpractice_Label']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling (optional but good for logistic regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [59]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_scaled, y_train)


LogisticRegression()

In [61]:
from sklearn.metrics import classification_report, confusion_matrix

# Choose appropriate input: scaled for LR, unscaled for RF
X_test_input = X_test_scaled if isinstance(model, LogisticRegression) else X_test

y_pred = model.predict(X_test_input)

print("📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\n📋 Classification Report:\n", classification_report(y_test, y_pred))


📊 Confusion Matrix:
 [[2 0]
 [0 2]]

📋 Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



In [63]:
import joblib

joblib.dump(model, 'malpractice_model.pkl')
joblib.dump(scaler, 'scaler.pkl')  # Save scaler if used (Logistic Regression)


['scaler.pkl']

In [65]:
# Apply model
X_all = df[['CA_Score', 'Mock_Score', 'Final_Score', 'External_Score']]
X_input = scaler.transform(X_all)

df['Prediction'] = model.predict(X_input)
df['Prediction_Prob'] = model.predict_proba(X_input)[:, 1]  # probability of malpractice


In [63]:
df.to_sql('ExamPredictions', engine, if_exists='replace', index=False)


20

In [93]:
pip install streamlit joblib numpy scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [85]:
pip install streamlit


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [67]:
pip show streamlit

Name: streamlit
Version: 1.37.1
Summary: A faster way to build and share data apps
Home-page: https://streamlit.io
Author: Snowflake Inc
Author-email: hello@streamlit.io
License: Apache License 2.0
Location: C:\ProgramData\anaconda3\Lib\site-packages
Requires: altair, blinker, cachetools, click, gitpython, numpy, packaging, pandas, pillow, protobuf, pyarrow, pydeck, requests, rich, tenacity, toml, tornado, typing-extensions, watchdog
Required-by: 
Note: you may need to restart the kernel to use updated packages.
